The purpose of this notebook is to calculate the power produced by a decay chain after the production of a given nuclide.

In [6]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"
endf8_decay_dir = cross_section_dir * "ENDF_Libraries-2\\ENDF-B-VIII.0\\decay\\"
jeff33_decay_dir = cross_section_dir * "ENDF_Libraries-2\\JEFF-3.3\\decay\\"
jendl5_decay_dir = cross_section_dir * "jendl5-dec_upd5\\"

"C:\\Cross-Section-Data\\jendl5-dec_upd5\\"

Now I will find the average decay energies for each nuclide. For the datasets I am using, the average decay energies are all given in keV. 

In [7]:
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];

Rename files in directory to be easier to parse by removing zero-padding.

In [37]:
function unzero_pad(num_str) 
    if (num_str == '0')
        return num_str
    end
    i = 1
    while i < length(num_str) && num_str[i] == '0'
        i += 1
    end
    return num_str[i : end]
end
function rename_files_in_dir(dir, cutoff_index)
    for f in readdir(dir)
        if tryparse(Int64, string(f[1])) == nothing
            mv(dir * f, dir * f[cutoff_index:end])
        end
        if f != join(unzero_pad.(split(f, "-")), "-")
            mv(dir * f, dir * join(unzero_pad.(split(f, "-")), "-"))
        end
    end
end
rename_files_in_dir(endf8_decay_dir, 12)
rename_files_in_dir(jeff33_decay_dir, 12)
rename_files_in_dir(jendl5_decay_dir, 4)

In [11]:
function get_decay_energy_from_file(file_as_arr, search_string)
    line = [l for l in file_as_arr if length(l) > length(search_string)
             && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))
        return join(split(line, " ", keepempty = false)[1])
    else 
        return "Energy not found"
    end
end

get_decay_energy_from_file (generic function with 1 method)

In [41]:
dir_search_string_dict = Dict([
    jeff33_decay_dir => Dict(["alpha" => "ALPHA ENERGY                   ="]),
    endf8_decay_dir => Dict(["alpha" => "Mean Alpha Energy:"]),
    jendl5_decay_dir => Dict(["alpha" => "Mean Alpha Energy",
    "beta-minus" => "Mean B- Energy", "gamma" => "Mean Gamma Energy",
    "neutron" => "Mean Neutron Energy", "proton" => "Mean Proton Energy"])  
])

Dict{String, Dict{String, String}} with 3 entries:
  "C:\\Cross-Section-Data\… => Dict("proton"=>"Mean Proton Energy", "alpha"=>"M…
  "C:\\Cross-Section-Data\… => Dict("alpha"=>"Mean Alpha Energy:")
  "C:\\Cross-Section-Data\… => Dict("alpha"=>"ALPHA ENERGY                   =")

In [12]:
function find_avg_decay_energy(nuclide, decay_type, dir)
    a, el = get_mass_name(nuclide)
    el = string(el)
    z = [i for i in 1:length(element_symbols) if element_symbols[i] == el][1]
    file_name = string(z) * "-" * el * "-" * string(a) * ".dat"
    file_path = dir * file_name
    file_as_array = split(open(f->read(f, String), file_path), "\n")
    search_string = dir_search_string_dict[dir][decay_type]
    return get_decay_energy_from_file(file_as_array, search_string)
end

find_avg_decay_energy (generic function with 1 method)

Now for some examples. Again, all average decay energies are given in keV. 

In [39]:
out = find_avg_decay_energy("238U", "alpha", jeff33_decay_dir)

"4187.0737"

In [43]:
out = find_avg_decay_energy("3H", "beta-minus", jendl5_decay_dir)

"5.69"

In [48]:
avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jendl5_decay_dir)

"Energy not found"

In [47]:
avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jendl5_decay_dir)
if avg_decay_energy == "Energy not found"
    avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jeff33_decay_dir)
end

"45.9500"

C:\Cross-Section-Data\ENDF_Libraries-2\ENDF-B-VIII.0\decay

In [4]:
decay_chain_dir = parent_dir * "ExportedData\\all_beta_decay_chains\\"
decay_chain_files = readdir(decay_chain_dir)
all_decay_chains = [CSV.read(decay_chain_dir * file, DataFrame)
                    for file in decay_chain_files]
all_decay_chains = [rename!(df, :Column1 => :Parent) for df in all_decay_chains]

3192-element Vector{DataFrame}:
 4×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Ag                   173.989                         5.69  100Cd
   2 │ 100Cd                    70.8363                        5.69  100In
   3 │ 100In                     8.10795                       5.69  100Sn
   4 │ 100Sn                     1.70238                       5.69  100Sb
 3×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Cd                    70.8363                        5.69  100In
   2 │ 100In                     8.10795                       5.69  100Sn
   3 │ 100Sn  

I derived the decay rate of the n-th generation nuclide in a decay chain here:

https://www.overleaf.com/project/627613def6b848465d85e5bb

https://github.com/MarcosP7635/Math-for-Energy/blob/main/main.tex

file:///C:/Users/engin/Downloads/Math_for_Energy(4).pdf 

In [4]:
factor(t, e_folding_time) = 1 - exp(t / -e_folding_time)

factor (generic function with 1 method)

In [5]:
a_prod(e_folding_times, t) = prod([factor(t, e_folding_times[k])
                                   for k in 2:length(e_folding_times)])

a_prod (generic function with 1 method)

In [6]:
r(e_fold_times, t) = (a_prod(e_fold_times, t) * exp(t / -e_fold_times[1])
                      / e_fold_times[1])


r (generic function with 1 method)

In [7]:
λ_0, λ_1, λ_2, t = @variables λ_0, λ_1, λ_2, t
expected_output = (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * exp(t / -λ_0) / λ_0

((1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [8]:
e_folding_times = [λ_0, λ_1, λ_2]
r(e_folding_times, t)
Test.@test r(e_folding_times, t) - expected_output == 0

Test Passed
  Expression: r(e_folding_times, t) - expected_output == 0
   Evaluated: 0 == 0

It works! :) 

The average beta decay energy is given in keV. 

In [9]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))

337//2103388558093277156

Avogadro's number

In [10]:
N_A = 6.02214076e23
conversion_factor = Rational(joules_per_keV * N_A)

6475021031458043//67108864

Now we calculate the power in watts

In [11]:
power_per_mole(decay_rate, decay_energy) = decay_rate * decay_energy * conversion_factor

power_per_mole (generic function with 1 method)

In [12]:
power_per_mole(e_folding_times, decay_energies, t) = sum([
    power_per_mole(r(e_folding_times[1:i], t), decay_energies[i])
    for i in 1:length(decay_energies)])

power_per_mole (generic function with 2 methods)

In [13]:
E_0, E_1, E_2, t = @variables E_0, E_1, E_2, t
expected_output = conversion_factor * sum([
    E_0 * exp(t / -λ_0) / λ_0,
    (1 - exp(t / -λ_1)) * E_1 * exp(t / -λ_0) / λ_0,
    (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * E_2 * exp(t / -λ_0) / λ_0
])

((6475021031458043//67108864)*E_0*exp(t / (-λ_0))) / λ_0 + ((6475021031458043//67108864)*E_1*(1 - exp(t / (-λ_1)))*exp(t / (-λ_0))) / λ_0 + ((6475021031458043//67108864)*E_2*(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [14]:
decay_energies = [E_0, E_1, E_2]
Test.@test Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) 
            - expected_output) == 0

Test Passed
  Expression: Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) - expected_output) == 0
   Evaluated: 0 == 0

In [15]:
x = [power_per_mole(df[!,"e-Folding Time (seconds)"], df[!,"Average beta-decay energy"], 2)
for df in all_decay_chains];

It takes about .1 seconds to calculate the power per mole for the entire dataframe for a given time. 

In [16]:
exponents_for_time_array = -10:20/1000:10
collect(exponents_for_time_array)

1001-element Vector{Float64}:
 -10.0
  -9.98
  -9.96
  -9.94
  -9.92
  -9.9
  -9.88
  -9.86
  -9.84
  -9.82
  -9.8
  -9.78
  -9.76
   ⋮
   9.78
   9.8
   9.82
   9.84
   9.86
   9.88
   9.9
   9.92
   9.94
   9.96
   9.98
  10.0

In [17]:
time_array = [BigFloat(10)^x for x in exponents_for_time_array]

1001-element Vector{BigFloat}:
 1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
 1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10
 1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10
 1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10
 1.202264434617413102533306847068551134853580025789314084263608408653746937282328e-10
 1.25892541179416618056939287182433126845996218444738646667433027351526766150798e-10
 1.318256738556404729589129642168660706457656431654852798000801526451019756623137e-10
 1.38038426460288664458362703761266589785155477230948719984049693714804016050564e-10
 1.445439770745927984903979184458196052506877843919927508765240153411912508988761e-10
 1.513561248436207171938059987874779303581263483786977823789873399913939914213657e-10
 1.584893192461110892181946192724010648088344291738932269688212001583469968929013e-10
 1.659586907437563078013118

In [18]:
power_time_series = [[power_per_mole(df[!,"e-Folding Time (seconds)"], 
                    df[!,"Average beta-decay energy"], time)
                    for time in time_array] for df in all_decay_chains]; 
zero_gen_isotopes = [string(df[1, "Parent"]) for df in all_decay_chains]
power_time_series_dict = Dict([])
stop = length(power_time_series)
for i in 1:stop
        power_time_series_dict[zero_gen_isotopes[i]] = power_time_series[i]
end
power_time_series_dict

Dict{Any, Any} with 3192 entries:
  "113Sn" => BigFloat[19.1026, 19.1026, 19.1026, 19.1026, 19.1026, 19.1026, 19.…
  "243Cf" => BigFloat[587251.0, 587251.0, 587251.0, 587251.0, 587251.0, 587251.…
  "168Eu" => BigFloat[1.90269e+09, 1.90269e+09, 1.90269e+09, 1.90269e+09, 1.902…
  "162Sm" => BigFloat[1.4094e+08, 1.4094e+08, 1.4094e+08, 1.4094e+08, 1.4094e+0…
  "11N"   => BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0,…
  "57V"   => BigFloat[1.08725e+09, 1.08725e+09, 1.08725e+09, 1.08725e+09, 1.087…
  "208Po" => BigFloat[4.16099, 4.16099, 4.16099, 4.16099, 4.16099, 4.16099, 4.1…
  "174Ir" => BigFloat[4.81695e+07, 4.81695e+07, 4.81695e+07, 4.81695e+07, 4.816…
  "144Gd" => BigFloat[1.41886e+06, 1.41886e+06, 1.41886e+06, 1.41886e+06, 1.418…
  "80Sr"  => BigFloat[59664.3, 59664.3, 59664.3, 59664.3, 59664.3, 59664.3, 596…
  "33Ar"  => BigFloat[2.19965e+09, 2.19965e+09, 2.19965e+09, 2.19965e+09, 2.199…
  "220Pa" => BigFloat[4.48174e+14, 4.48197e+14, 4.48221e+14, 4.48246e+14, 4

In [19]:
function plot_power_time_series(nuclide, time_array)
    power_time_series = Float64.(power_time_series_dict[nuclide]) * -1
    time_array = Float64.(time_array)
    plot(scatter(x = time_array, y = power_time_series, mode = "markers"), 
    Layout(xaxis_type = "log"))#, yaxis_type = "log"))
end
plot_power_time_series("207Bi", time_array)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

In [20]:
values(power_time_series_dict)

ValueIterator for a Dict{Any, Any} with 3192 entries. Values:
  BigFloat[19.10255687989461236685044743340581239013755137158583044737971404894…
  BigFloat[587251.3417034893162085043426417597518662359998193764706657792645493…
  BigFloat[1.902694346497644245150516668616150204766859907155112602001096115937…
  BigFloat[1.409403220047922261847711779732440399370103109599795278937175528118…
  BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, …
  BigFloat[1.087253912424558971525400118494233169459749268892055916591742346843…
  BigFloat[4.160990031372746078034922899154623194906258702410810853183757204621…
  BigFloat[4.816947714556571279120306684220982236639560494868173793622995720059…
  BigFloat[1.418862302097725720321943105981181420127809995803572798046172609174…
  BigFloat[59664.29435929451363606820111435521224661824346134398145632086382050…
  BigFloat[2.199646643180580411020156654991285400502149815425932111723296924897…
  BigFloat[4.4817436926458402547613507994885068

In [21]:
power_matrix = Matrix{Union{Float64, BigFloat}}(undef, stop, length(time_array))
for row in 1:stop
    power_matrix[row, :] = power_time_series_dict[zero_gen_isotopes[row]]
end
power_df = DataFrame(power_matrix, string.(time_array))

,1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
,Union…
1,3.155380343474517773182881336006113448165850785347324068054552557712315167057752e+06
2,7.750282473052230410131589151789001926210682420581984528770889009429861047264477e+06
3,6.771154260504771926121105292668354778399529497285304976772711638720867902521081e+07
4,3.171157231156666882975197464330423674578587447320415567899638881508334689581838e+10
5,1.705593933659425828895044885609053134811681637366715319113371576457134049742354e-18
6,2.53692579603415792778394416593882461800050762529806888346674704412560119165307e+08
7,509.6409195010121875533388457038160623571648639965189717384238721768647758216825
8,7.417911677174506856344579947799272975701897108371049300218245589944594604179178e+09
9,289378.2804519037072109144206399175771913850710015473516742436811413904833393907


In [22]:
power_df[!, "Parent Isotope"] = zero_gen_isotopes

3192-element Vector{String}:
 "100Ag"
 "100Cd"
 "100In"
 "100Kr"
 "100Mo"
 "100Nb"
 "100Pd"
 "100Rb"
 "100Rh"
 "100Sn"
 "100Sr"
 "100Tc"
 "100Y"
 ⋮
 "99Pd"
 "99Rb"
 "99Rh"
 "99Sn"
 "99Sr"
 "99Tc"
 "99Y"
 "99Zr"
 "9B"
 "9C"
 "9He"
 "9Li"

In [23]:
CSV.write(cross_section_dir * "Julia_beta_power_time_series.csv", power_df)

"C:\\Cross-Section-Data\\Julia_beta_power_time_series.csv"

https://drive.google.com/file/d/1aOk0XagAuHJ9FUSMbESXMMc5aSb0LOwH/view?usp=sharing

Now to find the highest power isotopes as a function of time. 

In [24]:
str_time_array = string.(time_array)
max_power_isotopes = [zero_gen_isotopes[argmax(power_df[!, t])]
                    for t in str_time_array]
max_power_values = [power_df[argmax(power_df[!, t]), t] for t in str_time_array]

1001-element Vector{BigFloat}:
  1.995177526057844635471646285324118334960892132242675305370219098971134848221332e+18
  1.995155387422343567201226617517561047406349936153226565045696371027876121828926e+18
  1.995132205688386011442372507257589687917616758763492771273755044493092244583548e+18
  1.995107931721654673984125684621797570740134606912889242877026635864950149719735e+18
  1.995082514074653131752332553485297363788479167330838261845547485878575118163003e+18
  1.995055898877925333933432170354231274433091868423127499639691126442421954094593e+18
  1.99502802972617124392666695360302246589695026981375241545128192470229806413689e+18
  1.99499884755902024562124196012237664994552283374025112207504423135787455414925e+18
  1.994968290536212979741750945851149893015087196159183709740310884281246923570497e+18
  1.994936293906930686192975068376267902653788614336202485755903819813458367978445e+18
  1.994902789872999180824012734118711125671920564083721200149506098831911343106934e+18
  1.9948677074

In [25]:
max_power_df = DataFrame(Time_seconds = time_array, Power_watts = max_power_values,
Isotope = max_power_isotopes, stringtype = string)

,Time_seconds
,BigFloat
1,1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
2,1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10
3,1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10
4,1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10
5,1.202264434617413102533306847068551134853580025789314084263608408653746937282328e-10
6,1.25892541179416618056939287182433126845996218444738646667433027351526766150798e-10
7,1.318256738556404729589129642168660706457656431654852798000801526451019756623137e-10
8,1.38038426460288664458362703761266589785155477230948719984049693714804016050564e-10
9,1.445439770745927984903979184458196052506877843919927508765240153411912508988761e-10


In [26]:
plot(scatter(max_power_df, x=:Time_seconds, y=:Power_watts, mode="markers",
text=:Isotope, color=:Isotope),
Layout(xaxis_type = "log", yaxis_type = "log", 
color = max_power_df[!, "Isotope"], show_scale =true))

data: [
  "scatter with fields color, mode, text, type, x, and y"
]

layout: "layout with fields color, margin, show, template, xaxis, and yaxis"

In [27]:
uconvert(@u_str("J"), 16 * @u_str("MeV") * N_A)

1.5437653139729602e12 J

In [28]:
decay_chain_energies = [sum(df[!, "Average beta-decay energy"])
                        for df in all_decay_chains]
maximum(decay_chain_energies) * conversion_factor

1.8561616629085276e12

In [29]:
best_decay_chain_index = argmax(decay_chain_energies)
zero_gen_isotopes[best_decay_chain_index]

"218Tl"

In [30]:
decay_chain_energy_df = DataFrame(Parent_Isotope = zero_gen_isotopes, 
Energy_keV = decay_chain_energies)

,Parent_Isotope,Energy_keV
,String,Float64
1,100Ag,22.76
2,100Cd,17.07
3,100In,11.38
4,100Kr,45.52
5,100Mo,11.38
6,100Nb,17.07
7,100Pd,25.15
8,100Rb,39.83
9,100Rh,349.15


In [31]:
CSV.write(parent_dir * "ExportedData\\Beta_decay_chain_energies.csv", decay_chain_energy_df)

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\Beta_decay_chain_energies.csv"